In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install bitsandbytes

In [ ]:
!pip install -U bitsandbytes transformers

In [ ]:
!pip install faiss-gpu-cu12

In [ ]:
import pandas as pd
import numpy as np
import torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
data_path = "/content/drive/MyDrive/"

In [ ]:
train = pd.read_csv(data_path+"train.csv" )
test = pd.read_csv(data_path+"test.csv" )
sample = pd.read_csv(data_path+"sample_submission.csv")

In [ ]:
def preprocess(df):
    df.replace('-', np.nan, inplace=True)
    df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
    df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
    df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
    df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
    df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
    df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]
    df['사고인지 시간'] = df['사고인지 시간'].str.split('-').str[0].str.strip()

    return df


train = preprocess(train)
test = preprocess(test)


In [ ]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"'{row['공사종류(중분류)']}' 공사 중 "
            f"'{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [ ]:
# 훈련 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"'{row['공사종류(중분류)']}' 공사 중 "
            f"'{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
#gpu 강제 버전
model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0}  # GPU 0번에 모델 전체를 강제로 올림
)

In [ ]:
import torch
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from tqdm import tqdm

# Train 데이터 준비
train_questions_prevention = combined_training_data['question'].tolist()
train_answers_prevention = combined_training_data['answer'].tolist()

train_documents = [
    f"Q: {q1}\nA: {a1}"
    for q1, a1 in zip(train_questions_prevention, train_answers_prevention)
]

# 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-nli"
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 벡터 스토어에 문서 추가
vector_store = FAISS.from_texts(train_documents, embedding)

# Retriever 정의
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.1,
    return_full_text=False,
    max_new_tokens=64,
    batch_size=16  # 배치 크기 지정 (GPU 메모리에 맞게 조정)
)

prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 답변을 핵심 내용만 요약하여 간략하게 작성하세요.
- 서론, 배경 설명 또는 추가 설명을 절대 포함하지 마세요.
- 다음과 같은 조치를 취할 것을 제안합니다: 와 같은 내용을 포함하지 마세요.

{context}

### 질문:
{question}

[/INST]
"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)


In [ ]:
tokenizer.padding_side = 'left'

In [ ]:
# 테스트 데이터 준비
test_questions = combined_test_data['question'].tolist()
batch_size = 32  # GPU 메모리에 맞게 조정

test_results = []
print("테스트 실행 시작... 총 테스트 샘플 수:", len(test_questions))

# 배치 단위로 처리
for i in tqdm(range(0, len(test_questions), batch_size), desc="처리 진행률"):
    batch_questions = test_questions[i:i + batch_size]

    # 배치로 문서 검색
    docs_batch = retriever.batch(batch_questions)  # 배치로 문서 검색

    # 검색된 문서에서 답변 추출
    for docs in docs_batch:
        # 문서 1개만 가져오므로 docs[0]의 답변 부분을 추출
        answer = docs[0].page_content.split("A: ")[1]  # "Q: 질문\nA: 답변" 형식 가정
        test_results.append(answer)

print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

In [ ]:
submission = pd.read_csv(data_path+"sample_submission.csv", encoding = 'utf-8-sig')
# 최종 결과 저장
submission.iloc[:,1] = test_results
submission.iloc[:,2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('submissionv2.csv', index=False, encoding='utf-8-sig')

In [ ]:
test_results